# 🔐 Sistema de Segurança por Voz com Reconhecimento de Emoções
Este notebook grava sua voz, reconhece o texto usando IBM Watson e detecta a emoção do seu tom de voz para autorizar ou negar acesso.

In [28]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [29]:
# 📦 Importações
import sounddevice as sd
from scipy.io.wavfile import write
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import uuid
import os
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [35]:
import pandas as pd

# Carregar o CSV
df_csv = pd.read_csv("emocoes_padrao.csv")

# Mostrar as primeiras linhas
print(df_csv.head())


     emoção                      frase
0     raiva          eu estou convicto
1     raiva         isso é inaceitável
2     raiva           não suporto mais
3  tristeza  estou me sentindo sozinho
4  tristeza           nada faz sentido


In [41]:
# 🔐 Configurações da API IBM Watson
API_KEY = '_0GwP85CeCaKSUKOSz008pCbyvbtFwEN1apk_BCBZUde'
URL = 'https://api.au-syd.speech-to-text.watson.cloud.ibm.com/instances/eeb1d325-cf7b-4cf3-babd-cb0e4f8ea11f'

authenticator = IAMAuthenticator(API_KEY)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url('https://api.au-syd.speech-to-text.watson.cloud.ibm.com/instances/eeb1d325-cf7b-4cf3-babd-cb0e4f8ea11f')

In [42]:
# 🧠 Função de Detecção de Emoção Simulada
def identificar_emocao_por_texto(texto):
    texto = texto.lower()
    if any(palavra in texto for palavra in ["socorro", "estou com medo", "ajuda", "não me machuca"]):
        return "medo"
    elif any(palavra in texto for palavra in ["não aguento", "você não me escuta", "irritar"]):
        return "raiva"
    elif any(palavra in texto for palavra in ["triste", "sozinho", "ninguém me entende"]):
        return "tristeza"
    elif any(palavra in texto for palavra in ["feliz", "consegui", "incrível"]):
        return "alegria"
    elif any(palavra in texto for palavra in ["ansioso", "nervoso", "e se"]):
        return "ansiedade"
    elif any(palavra in texto for palavra in ["te amo", "sinto sua falta", "você me faz bem"]):
        return "amor"
    else:
        return "neutro"


In [43]:
# 🎙️ Funções de Gravação e Reconhecimento
def gravar_audio(duracao=5, arquivo="voz.wav"):
    fs = 44100
    print("🎙️ Gravando...")
    audio = sd.rec(int(duracao * fs), samplerate=fs, channels=1)
    sd.wait()
    write(arquivo, fs, audio)
    print("✅ Gravação salva:", arquivo)

def reconhecer_voz_ibm(arquivo):
    with open(arquivo, 'rb') as audio_file:
        resposta = stt.recognize(
            audio=audio_file,
            content_type='audio/wav',
            model='pt-BR_BroadbandModel'
        ).get_result()
    try:
        texto = resposta['results'][0]['alternatives'][0]['transcript']
        return texto.strip().lower()
    except:
        return None

In [44]:
# Simulação de predição com modelo de emoção (aqui está fake, só para testar)
def extrair_emocao(arquivo):
    y, sr = librosa.load(arquivo, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc = np.mean(mfcc.T, axis=0)

    # Simulando emoção
    emocoes_fake = ['raiva', 'feliz', 'neutro', 'triste']
    return np.random.choice(emocoes_fake)


In [45]:
# 🔒 Sistema de Segurança
def sistema_seguranca():
    nome_arquivo = f"voz_{uuid.uuid4().hex}.wav"
    gravar_audio(5, nome_arquivo)
    
    texto = reconhecer_voz_ibm(nome_arquivo)
    emocao = extrair_emocao(nome_arquivo)

    print(f"\n🗣️ Texto reconhecido: {texto}")
    print(f"🧠 Emoção detectada: {emocao}")

    if texto and "acesso autorizado" in texto:
        if emocao in ['calmo', 'feliz']:
            print("✅ Acesso liberado.")
        else:
            print("⚠️ Frase correta, mas emoção suspeita.")
    else:
        print("❌ Acesso negado.")
    
    os.remove(nome_arquivo)

In [54]:
# ▶️ Executar o sistema
sistema_seguranca()

🎙️ Gravando...
✅ Gravação salva: voz_179701d972fe4c6694144078482933e5.wav

🗣️ Texto reconhecido: é um pouco cedo antes de
🧠 Emoção detectada: raiva
❌ Acesso negado.
